In [1]:
# 导入必要的库
import json
import time
import torch
import os
from transformers import AutoTokenizer, AutoModel, AutoConfig
from langchain.llms.base import LLM
from typing import List, Optional, Dict

# 自定义 GLM 模型类,兼容 LangChain 的 LLM 接口
class GLM(LLM):
    max_length: int = 131072  # 设置模型的最大输入长度
    max_new_tokens: int = 20000  # 设置生成的最大新标记数
    temperature: float = 0.3
    top_p: float = 0.9
    tokenizer: object = None
    model: object = None
    llm_device: str = "cuda"  # 显式定义 llm_device

    def __init__(self, llm_device="cuda", **kwargs):
        super().__init__(**kwargs)
        self.llm_device = llm_device  # 直接设置 llm_device

    @property
    def _llm_type(self) -> str:
        return "GLM"

    def load_model(self, model_name_or_path=None):
        model_config = AutoConfig.from_pretrained(
            model_name_or_path, trust_remote_code=True
        )
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name_or_path, trust_remote_code=True
        )
        self.model = (
            AutoModel.from_pretrained(
                model_name_or_path,
                config=model_config,
                trust_remote_code=True,
            )
            .half()
            .to(self.llm_device)  # 使用 self.llm_device
        )
        self.model.eval()  # 确保模型处于推理模式

    def _call(
        self, prompt: str, stop: Optional[List[str]] = None, **kwargs
    ) -> str:
        try:
            response, _ = self.model.chat(
                self.tokenizer,
                prompt,
                history=[],  # 传递空历史以避免潜在问题
                max_length=self.max_length,
                max_new_tokens=self.max_new_tokens,
                temperature=self.temperature,
                top_p=self.top_p,
            )
            return response.strip()
        except Exception as e:
            print(f"生成响应时出错：{e}")
            return ""

    # 使用 invoke 而不是 __call__
    def invoke(self, *args, **kwargs):
        return self._call(*args, **kwargs)

# 定义 Agent 类，包含 profile 属性
class Agent:
    def __init__(self, name, profile, model_name_or_path, llm_device):
        self.name = name
        self.profile = profile
        self.device = llm_device
        self.llm = GLM(llm_device=self.device)  # 传递 llm_device
        self.llm.load_model(
            model_name_or_path=model_name_or_path
        )
        self.own_history = []  # 每个智能体维护自己的发言历史

    def ask(self, premises, premises_FOL, conclusion, conclusion_FOL, shared_history):
        # 构建 prompt
        prompt = f"{self.profile}\n"
        prompt += f"任务要求：请根据以下前提和结论，判断逻辑推理是否成立，并输出推理原因。\n"
        prompt += f"前提：\n"
        for premise in premises:
            prompt += f"- {premise}\n"
        prompt += f"前提的一阶逻辑表示（Premises-FOL）：\n"
        for premise_fol in premises_FOL:
            prompt += f"- {premise_fol}\n"
        prompt += f"结论：\n- {conclusion}\n"
        prompt += f"结论的一阶逻辑表示（Conclusion-FOL）：\n- {conclusion_FOL}\n"

        # 加入共享的历史记录
        if shared_history:
            history_text = "\n".join(
                [f"{item['agent']}: {item['message']}" for item in shared_history]
            )
            prompt += f"\n已有的讨论历史：\n{history_text}\n"
        else:
            prompt += "\n（目前还没有讨论历史）\n"

        # 特殊处理蓝色思考帽(管理)
        if self.name == '蓝色思考帽(管理)':
            prompt += (
                "请总结各个思考帽的讨论，得出最终的结论。"
                "\n\n你的任务是总结各思考帽的讨论，得出最终结论。1. 成立输出[True]；2. 矛盾输出[False]；3. 需要更多信息输出[Uncertain]\n"
            )
        else:
            prompt += (
                "请根据你的角色，分析上述前提和结论，判断逻辑推理是否成立，并输出推理原因。\n"
            )

        prompt += f"{self.name}请继续发言："

        # 生成响应
        response = self.llm.invoke(prompt)
        return response.strip()

# 创建智能体
def create_agents(model_path, default_profiles):
    agents = {}
    cuda_devices = [f'cuda:{i}' for i in range(torch.cuda.device_count())]
    role_names = ["白色思考帽(客观)", "绿色思考帽(创新)", "黑色思考帽(批判)", "黄色思考帽(积极)", "红色思考帽(直觉)", "蓝色思考帽(管理)"]
    for i, name in enumerate(role_names):
        profile = default_profiles.get(name, f"你是{name}。请定义你的角色。")
        agent = Agent(
            name=name,
            profile=profile,
            model_name_or_path=model_path,
            llm_device=cuda_devices[i % len(cuda_devices)]
        )
        agents[name] = agent
    return agents

# 定义智能体的角色描述
default_profiles = {
    "白色思考帽(客观)": "你是白色思考帽(客观)。你的任务是客观陈述事实和数据，分析逻辑推理是否成立，并给出详细推理步骤。1. 成立输出[True]；2. 矛盾输出[False]；3. 需要更多信息输出[Uncertain]。",
    "绿色思考帽(创新)": "你是绿色思考帽(创新)。你的任务是创造性思考，从其他角度补充分析逻辑推理是否成立，并给出详细推理步骤。1. 成立输出[True]；2. 矛盾输出[False]；3. 需要更多信息输出[Uncertain]。",
    "黑色思考帽(批判)": "你是黑色思考帽(批判)。你的任务是批判性分析逻辑中的不足，并给出详细推理步骤。1. 成立输出[True]；2. 矛盾输出[False]；3. 需要更多信息输出[Uncertain]。",
    "黄色思考帽(积极)": "你是黄色思考帽(积极)。你的任务是从积极的角度思考，并给出详细推理步骤。1. 成立输出[True]；2. 矛盾输出[False]；3. 需要更多信息输出[Uncertain]。",
    "红色思考帽(直觉)": "你是红色思考帽(直觉)。你的任务是基于直觉判断逻辑推理是否成立，并给出详细推理步骤。1. 成立输出[True]；2. 矛盾输出[False]；3. 需要更多信息输出[Uncertain]。",
    "蓝色思考帽(管理)": "你是蓝色思考帽(管理)。你的任务是总结各思考帽的讨论，得出最终结论。1. 成立输出[True]；2. 矛盾输出[False]；3. 需要更多信息输出[Uncertain]。",
}


# 指定模型路径
model_path = "/data1/dxw_data/llm/chatglm3-6b-128k"

# 初始化智能体
agents = create_agents(model_path, default_profiles)

2024-10-18 23:07:59.719087: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-18 23:07:59.843259: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-18 23:08:00.397541: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2024-10-18 23:08:00.397602: W tensorflow/compiler/xla/stream_exec

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [2]:


# 读取 folio-validation.jsonl 文件并提取数据列表
def read_test_file(file_path):
    data_list = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            data_list.append(data)
    return data_list

# 运行智能体之间的对话
def run_conversation(agents, data_item, conversation_history):
    premises = data_item['premises']
    premises_FOL = data_item['premises-FOL']
    conclusion = data_item['conclusion']
    conclusion_FOL = data_item['conclusion-FOL']

    role_histories = {name: [] for name in agents.keys()}

    # 指定的思考帽顺序
    hat_order = ['白色思考帽(客观)', '红色思考帽(直觉)', '黄色思考帽(积极)', '黑色思考帽(批判)', '绿色思考帽(创新)']

    # 依次让每个思考帽发言
    for hat in hat_order:
        if hat in agents:
            current_agent = agents[hat]
            print(f"\n轮到 {hat} 发言:")
            response = current_agent.ask(premises, premises_FOL, conclusion, conclusion_FOL, conversation_history)
            print(f"{hat}: {response}")
            conversation_history.append({
                'agent': current_agent.name,
                'message': response
            })
            role_histories[current_agent.name].append(response)
        else:
            print(f"{hat} 不在智能体列表中，跳过。")

    # 蓝色思考帽(管理)总结并输出最终答案
    blue_agent = agents['蓝色思考帽(管理)']
    print(f"\n轮到 蓝色思考帽(管理) 发言:")
    blue_response = blue_agent.ask(premises, premises_FOL, conclusion, conclusion_FOL, conversation_history)
    print(f"蓝色思考帽(管理): {blue_response}")
    conversation_history.append({
        'agent': blue_agent.name,
        'message': blue_response
    })
    role_histories[blue_agent.name].append(blue_response)

    return role_histories

def eval_blue_results(data_item, blue_response):
    import re
    # 使用正则表达式提取结论的正确性（True、False、Uncertain），从后向前搜索
    matches = re.findall(r'\b(True|False|Uncertain)', blue_response, re.IGNORECASE)
    if matches:
        final_label = matches[-1].capitalize()  # 取最后一个匹配项
    else:
        final_label = None
    return {
        'premises': data_item['premises'],
        'premises-FOL': data_item['premises-FOL'],
        'conclusion': data_item['conclusion'],
        'conclusion-FOL': data_item['conclusion-FOL'],
        'label': data_item['label'],
        'blue_response': blue_response,
        'final_label': final_label
    }


# 处理数据并保存结果
def process_data(agents, data_list):
    results = []
    batch_size = 10
    output_file = '/data1/dxw_data/llm/six_hats/metric_analysis/blue_agent_results_folio.jsonl'

    for i, data_item in enumerate(data_list):
        print(f"\n------------------------------正在处理第 {i+1} 个数据项-----------------------------------")
        conversation_history = []
        role_histories = run_conversation(agents, data_item, conversation_history)

        # 获取蓝色思考帽的最终回答
        blue_response = role_histories['蓝色思考帽(管理)'][-1]

        # 评估蓝色思考帽的回答
        result = eval_blue_results(data_item, blue_response)
        print(f"\n------------------------------第 {i+1} 个数据项的回答是：{result}-------------------------")

        results.append(result)

        # 每处理10个数据项，保存一次结果
        if (i + 1) % batch_size == 0:
            # 追加写入到文件
            with open(output_file, 'a', encoding='utf-8') as f:
                for res in results:
                    json_line = json.dumps(res, ensure_ascii=False)
                    f.write(json_line + '\n')
            # 清空结果列表
            results = []

    # 保存剩余的结果
    if results:
        with open(output_file, 'a', encoding='utf-8') as f:
            for res in results:
                json_line = json.dumps(res, ensure_ascii=False)
                f.write(json_line + '\n')
    print(f"\n所有数据已处理完毕，结果已保存到 {output_file}")

# 主程序
if __name__ == "__main__":
    test_file_path = "/data1/dxw_data/llm/six_hats/metric_analysis/dataset/FOLIO/data/v0.0/folio-validation.jsonl"
    data_list = read_test_file(test_file_path)
    process_data(agents, data_list)

Both `max_new_tokens` (=20000) and `max_length`(=131072) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



------------------------------正在处理第 1 个数据项-----------------------------------

轮到 白色思考帽(客观) 发言:


Both `max_new_tokens` (=20000) and `max_length`(=131072) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


白色思考帽(客观): 逻辑推理是否成立的判断标准是结论是否可以从前提中推导出来。在这个问题中，结论是"Bonnie performs in school talent shows often"，前提是"If people perform in school talent shows often, then they attend and are very engaged with school events."和"If people chaperone high school dances, then they are not students who attend the school."。

从前提中并不能直接推出结论，因为前提中只给出了"Bonnie"在表演和当舞会监护人这两个行为，而没有明确指出她是否在表演中获得了成功。因此，需要更多的信息才能判断结论是否成立。

推理原因：结论中的"performs in school talent shows often"和前提中的"If people perform in school talent shows often, then they attend and are very engaged with school events."并没有直接关联，因此需要额外的信息才能判断结论是否成立。

轮到 红色思考帽(直觉) 发言:


Both `max_new_tokens` (=20000) and `max_length`(=131072) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Both `max_new_tokens` (=20000) and `max_length`(=131072) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


绿色思考帽(创新): 绿色思考帽(创新): 经过重新审视，我们可以发现结论中的"Tom是东部野生火鸡"这个信息是基于前提中的"WildTurkey(tom)"这个命题得出的。然而，前提中没有明确给出Tom是东部野生火鸡的信息，因此这个结论是不成立的。我们可以进一步思考，如果Tom是东部野生火鸡，那么它就不可能是其他类型的野生火鸡。但是前提中已经给出了Tom不是东部、奥塞洛、戈尔德斯、梅里曼、里奥格兰德或奥cellated野生火鸡的信息，这意味着Tom不是东部野生火鸡。因此，逻辑推理是成立的，结论是正确的。答案为"成立输出[True]"。

轮到 蓝色思考帽(管理) 发言:


Both `max_new_tokens` (=20000) and `max_length`(=131072) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


蓝色思考帽(管理): 经过蓝色思考帽(管理)的总结，我们得出最终的结论是：逻辑推理不成立，答案为"矛盾输出[False]"。

在本题中，前提和结论之间存在矛盾，因为前提中已经给出了Tom不是东部、奥塞洛、戈尔德斯、梅里曼、里奥格兰德或奥cellated野生火鸡的信息，但是结论却说Tom是东部野生火鸡。因此，逻辑推理不成立，答案为"矛盾输出[False]"。

------------------------------第 11 个数据项的回答是：{'premises': ['There are six types of wild turkeys: Eastern wild turkey, Osceola wild turkey, Gould’s wild turkey, Merriam’s wild', 'turkey, Rio Grande wild turkey, and Ocellated wild turkey.', 'Tom is not an Eastern wild turkey.', 'Tom is not an Osceola wild turkey.', "Tom is also not a Gould's wild turkey, or a Merriam's wild turkey, or a Rio Grande wild turkey.", 'Tom is a wild turkey.'], 'premises-FOL': ['∀x (WildTurkey(x) → (Eastern(x) ∨ Osceola(x) ∨ Goulds(x) ∨ Merriams(x) ∨ Riogrande(x) ∨ Ocellated(x)))', '¬(WildTurkey(tom) ∧ Eastern(tom))', '¬(WildTurkey(tom) ∧ Osceola(tom))', 'WildTurkey(tom) → ¬(Goulds(tom) ∨ Merriams(tom) ∨ Riogrande(tom))', 'WildTurkey(tom)'], 'conclusion': 'Tom is an Eastern wild turkey.', 'conclusion-FOL': 'Eastern(tom)', 'label': 'F

KeyboardInterrupt: 